<a href="https://colab.research.google.com/github/mereilly/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()
print (f"Raw records count: {df.count()}")

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id',
                     'star_rating',
                     'helpful_votes',
                     'total_votes',
                     'vine',
                     'verified_purchase'])

vine_df.show()
print (f"Vine count: {vine_df.count()}")

In [ ]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes 
# count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having 
# division by zero errors later on.

filter_df = vine_df.filter(vine_df.total_votes >= 20)
filter_df.show()
print(f"Filter DF count: {filter_df.count()}")

In [ ]:
# Step 2: Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to 
# retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater 
# than 50%.

helpful_df = filter_df.filter(filter_df.helpful_votes/filter_df.total_votes >= 0.5)
helpful_df.show()
print(f"Helpful DF count: {helpful_df.count()}")

In [ ]:
# Step 3: Filter the DataFrame or table created in Step 2, and create a new DataFrame or table 
# that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'. (YES Paid)

vine_y_df = helpful_df.filter(helpful_df.vine == 'Y')
vine_y_df.show()


In [ ]:
# Step 4: Repeat Step 3, but this time retrieve all the rows where the review was not part of 
# the Vine program (unpaid), vine == 'N' (Not Paid).

vine_n_df = helpful_df.filter(helpful_df.vine == 'N')
vine_n_df.show()
print(f"Vine N DF count: {vine_n_df.count()}")

In [ ]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews, and the 
# percentage of 5-star reviews for the two types of review (paid vs unpaid).

vine_y_total = vine_y_df.count()
vine_n_total = vine_n_df.count()

vine_y_five_star = vine_y_df.filter(vine_y_df.star_rating == 5).count()
vine_n_five_star = vine_n_df.filter(vine_n_df.star_rating == 5).count()

vine_y_five_star_pct = (vine_y_five_star/vine_y_total) * 100
vine_n_five_star_pct = (vine_n_five_star/vine_n_total) * 100

print (f"Five Stars in Vine Y: {vine_y_five_star}, Vine Y Total Reviews: {vine_y_total}, Portion that were five star reviews: {vine_y_five_star_pct:.1f}")
print (f"Five Stars in Vine X: {vine_n_five_star}, Vine N Total Reviews: {vine_n_total}, Portion that were five star reviews: {vine_n_five_star_pct:.1f}")


Five Stars in Vine Y: 10, Vine Y Total Reviews: 21, Portion that were five star reviews: 47.6
Five Stars in Vine X: 3448, Vine N Total Reviews: 6690, Portion that were five star reviews: 51.5
